# PHAS1240 - Reading Week Task

# Student Number: 14009415

#### The task below will attempt to plot two lines of least squares fit (unweighted vs weighted by the inverse of uncertainty squared). The data used to accomplish this will be from a student experiment to calculate the specific charge (charge to mass ratio) of an electron. 

The force experienced by a moving charge in a magnetic field is: 

$$ F_r = evB = \frac{mv^2}{r} $$

Where r is the radius of orbit.

The speed of the electrons is given by the following: 

$$  \frac{1}{2} m v^2 = e V $$

Rearranging these equations for r provides us with: 

$$  r = \frac{1}{B} \sqrt{ \frac{2m}{e}} \sqrt{V}$$

We will be plotting $ \sqrt{Voltage} $ against radius (r) on our graphs and measuring the gradients of best fit lines to find the values of specific charge. 

## Extracting Data

##### The code below extracts the data from our data CSV file, and uses a comma as a delimiter. This means that the comma is used to identify elements as seperate from one another. 

Upon extracting it unpacks it in to three arrays, this is done using the unpack=True command as seen below. 

It then square roots in a per element way the x-data array to ensure the x-axis is the $\sqrt(V)$

In [2]:
import numpy as np               #Import numpy
import matplotlib.pyplot as plt  #Import plotting library

#Ensure all graphs are displayed inline
%matplotlib inline               

#Unpack data to two arrays using comma as a
x1data, y1data, deltay1  = np.loadtxt("eovermdata2016.csv", delimiter=",",unpack=True)    

#Square root our V values to ensure our X axis will be square root of V
x1data = np.sqrt(x1data)

#Minimum and maximum data values of data set (for use when placing objects on the graph)
x_min=np.min(x1data)
y_min=np.min(y1data)
x_max=np.max(x1data)

FileNotFoundError: [Errno 2] No such file or directory: 'eovermdata2016.csv'

## Calculating unweighted gradient and intercept (and their uncertainties)
##### The code below will be calculating a gradient and intercept for an unweighted line of least squares fit. 

It will use the following formulae to calculate gradient and intercept: [1]

$$ m = \frac{\sum(x_i - \bar{x}) y_i}{\sum (x_i - \bar{x})^2} = \frac{\sum x_i (y_i - \bar{y})}{\sum x_i (x_i - \bar{x})} $$

$$ c = \bar{y} - m \bar{x} $$

Where $\bar{y}$ and $\bar{x}$ represent the mean of X values and mean of Y values respectively.

Uncertainty for gradient (m) and intercept (c) can be calculated using the following formulae: [1]

$$ \Delta m = \sqrt{\frac{1}{D} \frac{\sum{d_i^2}}{(n-2)}} $$

$$ \Delta c = \sqrt{ \left(\frac{1}{n} + \frac{\bar{x}^2}{D}\right) \frac{\sum{d_i^2}}{(n-2)}} $$

Where: $ d_i = y_i - mx_i - c $ and $ D =  \sum{x_i - \bar{x}} $ 

In [2]:
#Calculate x and y means
ymean = np.mean(y1data) 
xmean = np.mean(x1data)

#Initialise the variables we will use to calculate gradient and uncertainties
noOfDataValues = len(x1data)

#Calculating numerator and denomenator of gradient function. 
mnum = sum(x1data*(y1data-ymean))
mdenom = sum(x1data*(x1data-xmean))

#Calculate value of D (for use in uncertainty)
D = sum((x1data-xmean)**2)
    
#Calculating gradient and intercept
UW_gradient = mnum/mdenom
UW_intercept = ymean - (UW_gradient*xmean)

#Calculating sigma of di squared (for use in uncertainty equations)
sumOfDiSquared = sum(np.power((y1data-(UW_gradient*x1data)-UW_intercept),2))

#Calculate y points for line of best fit
UW_xdata = np.linspace(11,19,2)    #Create array for x values of line of best fit
UW_ydata = np.copy(UW_xdata)
UW_ydata = UW_gradient*UW_xdata+UW_intercept
    
#Calculating uncertainty in gradient
UW_gradUncertainty = np.sqrt(sumOfDiSquared/(D*(noOfDataValues-2)))

#Calculating uncertainty in intercept
UW_interceptUncertainty = np.sqrt(((1/noOfDataValues)+(xmean**2/D))*sumOfDiSquared/(noOfDataValues-2))

print()
print("The unweighted gradient at full precision is",UW_gradient, '±', UW_gradUncertainty, "m V^(-1/2)")
print("The unweighted intercept at full precision is", UW_intercept,'±', UW_interceptUncertainty, "m")
print()


The unweighted gradient at full precision is 0.00292367220595 ± 0.000356444318526 m V^(-1/2)
The unweighted intercept at full precision is -0.00333963522913 ± 0.00540574574025 m



## Calculating weighted gradient and intercept (and their uncertainties)
##### The code below will be calculating the gradient, intercept and their uncertainties for the weighted line of least squares fit. 

We will weight the data points as follows: (This will ensure that data points with greater uncertainty contribute less to our least squares line of best fit: [2]

$$ w_i = \frac{1}{(\Delta y_i)^2} $$

Our gradient function will be of the following form: [2]

$$  m = \frac{\sum_i w_i \sum_i w_i x_i y_i - \sum_i w_i x_i \sum_i w_i
y_i}{\sum_i w_i \sum_i w_i x_i^2 - \left(\sum_i w_i x_i \right)^2}\\
= \frac{\sum_i w_i \sum_i w_i x_i y_i - \sum_i w_i x_i \sum_i w_i
y_i}{\delta} $$

Our intercept function will be calculated with the following: [2]

$$ c = \frac{\sum_i w_i x_i^2 \sum_i w_i y_i - \sum_i w_i x_i \sum_i w_i x_i y_i}{\delta} $$

Uncertainty in gradient will be calculated as follows: [2]

$$ \Delta m = \sqrt{ \frac{\sum_i w_i}{\delta}} $$

And the uncertainty in intercept with: [2]

$$ \Delta c = \sqrt{ \frac{\sum_i x_i^2 w_i}{\delta}} $$

In [3]:
#We will first form the array containing W
w = 1/(np.square(np.copy(deltay1)))

#Now we will calculate lower-case delta to simplify calculations
delta = sum(w)*sum(w*(x1data**2))-(sum(w*x1data))**2

#Calculating our gradient function
W_gradient = ((sum(w)*sum(w*x1data*y1data))-(sum(w*x1data)*sum(w*y1data)))/delta

#Calculating our intercept function
W_intercept = (sum(w*(x1data**2))*sum(w*y1data) - sum(w*x1data)*sum(w*x1data*y1data))/delta

#Calculatin uncertainty in weighted gradient
W_gradUncertainty = np.sqrt(sum(w)/delta) 

#Calculating uncertainty in weighted intercept
W_interceptUncertainty = np.sqrt(sum((x1data**2)*w)/delta)

#Create x and y array for line of best fit
W_xdata = np.linspace(11,19,2)    #Create array for x values of line of best fit
W_ydata = np.copy(W_xdata)
W_ydata = W_gradient*W_xdata+W_intercept

print()
print("The weighted gradient at full precision is",W_gradient,'±', W_gradUncertainty, "m V^(-1/2)")
print("The weighted intercept at full precision is", W_intercept,'±', W_interceptUncertainty, "m")
print()


The weighted gradient at full precision is 0.00244923831368 ± 0.000536132407511 m V^(-1/2)
The weighted intercept at full precision is 0.00300020627965 ± 0.00790442503692 m



## Calculating specific charge and its uncertainty
##### The code below will calculate both the e/m ratio and its uncertainty for both the weighted and unweighted lines of least squares fit

Our formula for the graph is 

$$  r = \frac{1}{B} \sqrt{ \frac{2m}{e}} \sqrt{V} $$

Where $ \sqrt(V) $ is our y-axis, and r is our x-axis. 

Therefore: 

$$ Gradient = \frac{1}{B}\sqrt{ \frac{2m}{e}} $$

Therefore

$$ \frac{e}{m} = \frac{2}{Gradient^2B^2} $$

Using the general formulae for uncertainty, the uncertainty of this can be calculated as follows: [1]

$$ (\Delta Z)^2 = \left(\frac{\partial Z}{\partial A} \right)^2(\Delta A)^2 + \left(\frac{\partial Z}{\partial B} \right)^2(\Delta B)^2 $$

$$ \Delta \frac{e}{m} = \sqrt{\left( \frac{\partial \frac{e}{m}}{\partial Gradient}^2 * \Delta Gradient^2 \right)+ \left (\frac{\partial \frac{e}{m}}{\partial B}^2 * \Delta B^2 \right)} $$

Therefore 

$$ \Delta \frac{e}{m} = 2 * \sqrt{\frac{4*\Delta Gradient^2} {Gradient^6B^4} + \frac{4*\Delta B^2} {Gradient^4B^6}} $$ 


In [6]:
#Calculating e/m charge ratio for both weighted and unweighted lines of best fit
B = 1.28e-3
delB = 0.01e-3
UW_emRatio =2/((UW_gradient**2)*(B**2))
W_emRatio = 2/((W_gradient**2)*(B**2))

#Calculating uncertainty in e/m for both weighted and unweighted 
UW_emUncertainty = 2*np.sqrt((4*UW_gradUncertainty**2)/((UW_gradient**6)*(B**4)) + (4*delB**2)/((UW_gradient**4)*(B**6)))
W_emUncertainty = 2*np.sqrt((4*W_gradUncertainty**2)/((W_gradient**6)*(B**4)) + (4*delB**2)/((W_gradient**4)*(B**6)))

print()
print('The unweighted e/m value is' ,UW_emRatio, '±', UW_emUncertainty)
print('The weighted e/m value is' ,W_emRatio, '±', W_emUncertainty )
print()


The unweighted e/m value is 142808052991.0 ± 34892779360.7
The weighted e/m value is 203492293827.0 ± 89144674241.5



## Plotting the graph
##### Plotted below are the collected data values, in addition to both weighted and unweighted lines of best fit

In [1]:
#This sets the size of the figure/graph
plt.figure(figsize=(9,6))                                    

#Plot grid
plt.grid(True)

#Plot data, add label, and change linewidth of line                                                  
plt.plot(UW_xdata,UW_ydata,'b--',label="$\mathit{Line \  of \ least \ squares \ fit \ (Unweighted)}$", linewidth=1.25)        
plt.plot(W_xdata, W_ydata,'r-.',label="$\mathit{Line \  of \ least \ squares \ fit \ (Weighted)}$", linewidth=1.25)
plt.errorbar(x1data,y1data,yerr=deltay1,fmt='bo')
plt.plot(x1data,y1data, 'k.', label="$\mathit{Experimental \ Data \ Values}$", linewidth=0.5) 



#Add labels, title and legend
plt.ylabel('$\mathit{radius \ of \ orbit \ (metres)}$',fontsize=15)   #Add labels as Math Text using $ 
plt.xlabel('$\sqrt{voltage}} $ $/V^{0.5} $', fontsize=15)
plt.title('$ \mathrm{Deflection \ of \ an \ electron \ beam \ in \ a \ magnetic \ field}$',loc="center", fontsize=20)   #Add centred title as Roman font in Math Text 
plt.legend(loc="best")
plt.xlim([11,19])

#Add gradient, intercept and their uncertainties to the graph
plt.text(1.52*x_min, 1.5*y_min, "The unweighted gradient is {0:0.4f} ± {1:0.4f} m V^(-1/2)".format(UW_gradient,UW_gradUncertainty), style='italic', fontsize=9,  bbox={'facecolor':'white', 'alpha':1, 'pad':5})
plt.text(1.52*x_min,1.43*y_min, "The unweighted intercept is {0:0.3f} ± {1:0.3f} m".format(UW_gradient,UW_interceptUncertainty), style='italic', fontsize=9, bbox={'facecolor':'white', 'alpha':1, 'pad':5})
plt.text(1.52*x_min, 1.3*y_min, "The weighted gradient is {0:0.4f} ± {1:0.4f} m V^(-1/2)".format(W_gradient,W_gradUncertainty), style='italic', fontsize=9,  bbox={'facecolor':'white', 'alpha':1, 'pad':5})
plt.text(1.52*x_min,1.23*y_min, "The weighted intercept is {0:0.3f} ± {1:0.3f} m".format(W_gradient,W_interceptUncertainty), style='italic', fontsize=9, bbox={'facecolor':'white', 'alpha':1, 'pad':5})
plt.text(1.52*x_min, 1.1*y_min, "The unweighted e/m ratio is {0:.1e} ± {1:.0e} C/kg".format(UW_emRatio,UW_emUncertainty), style='italic', fontsize=9,  bbox={'facecolor':'white', 'alpha':1, 'pad':5})
plt.text(1.52*x_min,1.03*y_min, "The weighted e/m ratio is {0:.1e} ± {1:.0e} C/kg".format(W_emRatio,W_emUncertainty), style='italic', fontsize=9, bbox={'facecolor':'white', 'alpha':1, 'pad':5})

print()
print('Unweighted Line of Best Fit: y = ({0:0.4f} ± {1:0.4f})x + ({2:0.3f} ± {3:0.3f})'.format(UW_gradient,UW_gradUncertainty,UW_gradient,UW_interceptUncertainty))
print('Weighted Line of Best Fit: y = ({0:0.4f} ± {1:0.4f})x + ({2:0.3f} ± {3:0.3f})'.format(W_gradient,W_gradUncertainty,W_gradient,W_interceptUncertainty))
print()

NameError: name 'plt' is not defined

$$ Unweighted \ Line \ of \ Best \ Fit: y = (0.0029 ± 0.0004)x + (0.003 ± 0.005) $$
$$ Weighted \ Line \ of \ Best \ Fit: y = (0.0024 ± 0.0005)x + (0.002 ± 0.008) $$

## What e/m value would I consider more reliable?

The unweighted e/m value is 142808052991.0 ± 34892779360.7

The weighted e/m value is 203492293827.0 ± 89144674241.5

The accepted value for e/m is (1.75882002 ± 0.0000001) x 10^11 C/kg (CODATA Fundamental Constants 2014)

At first glance, the weighted e/m value appears to be a more accurate value for e/m, as it reduces the influence of data points with high uncertainty on the final value of e/m. And is also therefore closer to the accepted value. 

However, upon calculating the uncertainty in two values of e/m, it can be observed that the uncertainty in weighted e/m is many times larger than that of the unweighted value. This is due to the nature of the calculations required to compute the weighted value of e/m. 

#####  Reliability is the consistency of a measure upon repeated procedures. Therefore, while the accuracy of the weighted e/m value is likely greater, due to the fact that its precision is lower, the unweighted value has high precision and therefore greater reliability. 

## Bibliography

#### [1] PHAS1240 Experimental Methods and Data Analysis (Prof N. Skipper and Dr P. Jones) - Pages 10
#### [2] PHAS1240 Reading Week task 2016: Using a weighted  t to calculate e=m